
# Принципи на качествения код в Python

Съдържание:
- SOLID
- DRY & YAGNI
- Clean Code
- PEP8
- Pylint

Писането на код е само една част от това да си програмист. В живота на едно парче код, то по-често ще бъде прочитано, отколкото променяно. В нашият програмистки живот, по-често ще ни се налага да четем код, отколкото да пишем. Затова едно от ключовите ни умения като програмист е това да пишем качествен и четим код.

Съществуват някои широко-разпространени добри практики за писане на качествен код (а и архитектура). Ще разгледаме някои тях

## SOLID

SOLID е колекция от принципи, имащи за цел да направят нашия (обектно-ориентиран) код по-четим, по-лесен за поддържане и по-гъвква. SOLID е акроним на 5 принципа, които ще разгледаме по-долу - **S**ingle-reponsibility principle, **O**pen-closed principle, **L**iskov substitution principle, **I**nterface segregation principle и **D**ependency inversion principle.

### Single-reponsibility

Както името му подсказва, single-reponsibility принципа казва, че "един клас не трябва да има повече от една причина да се променя". На по-прост език, това означава, че един клас трябва да има една отговорност/дейност.

Тази идея може да бъде разширена и към всички други програмни единици, не само класовете. Трябва да се стараем да пишем функции или класове, които правят едно нещо.

Нека разгледаме следната задача (source: [Advent of Code 2022, day 18, pt1](https://adventofcode.com/2022/day/18)):

Получаваме списък от координати на кубчета в тримерното пространство. На изхода на програмата трябва да върнем броя на "видимите" страни на кубчетата - например, ако имаме едно кубче с координати (1, 1, 1), и второ кубче с координати (2, 1, 1), то броя на видимите страни е 10 (две от страните няма да се виждат, защото са долепени една до друга).

Как бихме написали кода за тази задача ?


In [7]:
def solution():
    line = input()
    lines = []
    while line != '':
        lines.append(line)
        line = input()

    cubes = []
    for line in lines:
        x, y, z = line.split(',')
        cubes.append((int(x), int(y), int(z)))

    sides = 0

    for source_cube in cubes:
        hidden_sides = 0
        for target_cube in cubes:
            diff_x = abs(source_cube[0] - target_cube[0])
            diff_y = abs(source_cube[1] - target_cube[1])
            diff_z = abs(source_cube[2] - target_cube[2])

            if (diff_x == 1 and diff_y == 0 and diff_z == 0) or (diff_x == 0 and diff_y == 1 and diff_z == 0) or (diff_x == 0 and diff_y == 0 and diff_z == 1):
                hidden_sides += 1

        sides += 6 - hidden_sides
    
    print(sides)

solution()

10


Очевидно горната функция `solution` прави много неща - какво правим, ако имаме бъг в решението ? Трябва да прегледаме целия код, което тук е само 30 реда, но представете си, че говорим за 300 реда. Също така, ако направим промяна, не е много сигурно, че няма да счупим някое друго парче код. Затова нека започнем прилагането на принципа за единстветана отговорност.

Първото нещо, което ни хрумва да направим, е да отделим входа, сметките и изхода в отделни функции.

In [8]:
def handle_input() -> list[tuple[int, int, int]]:
    line = input()
    lines = []
    while line != '':
        lines.append(line)
        line = input()

    cubes = []
    for line in lines:
        x, y, z = line.split(',')
        cubes.append((int(x), int(y), int(z)))
    
    return cubes

def solution(cubes: list[tuple[int, int, int]]):
    sides = 0

    for source_cube in cubes:
        hidden_sides = 0
        for target_cube in cubes:
            diff_x = abs(source_cube[0] - target_cube[0])
            diff_y = abs(source_cube[1] - target_cube[1])
            diff_z = abs(source_cube[2] - target_cube[2])

            if (diff_x == 1 and diff_y == 0 and diff_z == 0) or (diff_x == 0 and diff_y == 1 and diff_z == 0) or (diff_x == 0 and diff_y == 0 and diff_z == 1):
                hidden_sides += 1

        sides += 6 - hidden_sides
    
    return sides

def handle_output(sides: int):
    print(f'The answer is {sides}')

cubes = handle_input()
result = solution(cubes)
handle_output(result)

The answer is 10


Една идея по-добре. Но все още нашите функции `handle_input` и `solution` правят повече от едно неща.

Нека разгледаме в детайли `handle_input` - макар и кратка, тя прави две основни неща - приема входа от клавиатурата, и го конвертира от низ до наредена тройка от цели числа. Какво би станало, ако искаме вместо от клавиатурата, нашия вход да се чете от файл ? Или пък от графичен интерфейс ? Тази промяна не би трябвало да има общо с това как превръщаме низ към наредена тройка. Затова ще разделим `handle_input` на две по-малки функции.

In [ ]:
def read_input() -> list[str]:
    line = input()
    lines = []
    while line != '':
        lines.append(line)
        line = input()

    return lines

def transform_input(lines: list[str]) -> list[tuple[int, int, int]]:
    cubes = []
    for line in lines:
        x, y, z = line.split(',')
        cubes.append((int(x), int(y), int(z)))
    
    return cubes

def solution(cubes: list[tuple[int, int, int]]):
    sides = 0

    for source_cube in cubes:
        hidden_sides = 0
        for target_cube in cubes:
            diff_x = abs(source_cube[0] - target_cube[0])
            diff_y = abs(source_cube[1] - target_cube[1])
            diff_z = abs(source_cube[2] - target_cube[2])

            if (diff_x == 1 and diff_y == 0 and diff_z == 0) or (diff_x == 0 and diff_y == 1 and diff_z == 0) or (diff_x == 0 and diff_y == 0 and diff_z == 1):
                hidden_sides += 1

        sides += 6 - hidden_sides
    
    return sides

def handle_output(sides: int):
    print(f'The answer is {sides}')


lines = read_input()
cubes = transform_input(lines)
result = solution(cubes)
handle_output(result)

Така вече, ако се наложи промяна в начина по който **четем** входа, няма да се налага да променяме функцията, която държи и логиката за трансформирането на входа. Или ако пък се промени формата на входа, трябва да променим само `transform_input`, без да пипаме `read_input`.

Друг плюс от това разделение е, че функцията `read_input` вече не е обвързана по никакъв начин с конкретната задача - спокойно тя може да бъде преизползвана за решаването на други задачи, които изискват четене от клавиатурата.

Можем обаче да отидем една стъпка по-напред - нека отделим логиката за трансформиране на само един ред, отделно.

In [9]:
def read_input() -> list[str]:
    line = input()
    lines = []
    while line != '':
        lines.append(line)
        line = input()

    return lines

def line_to_tuple(line: str) -> tuple[int, int, int]:
    x, y, z = line.split(',')
    return int(x), int(y), int(z)

def transform_input(lines: list[str]) -> list[tuple[int, int, int]]:
    return [line_to_tuple(line) for line in lines]

def solution(cubes: list[tuple[int, int, int]]):
    sides = 0

    for source_cube in cubes:
        hidden_sides = 0
        for target_cube in cubes:
            diff_x = abs(source_cube[0] - target_cube[0])
            diff_y = abs(source_cube[1] - target_cube[1])
            diff_z = abs(source_cube[2] - target_cube[2])

            if (diff_x == 1 and diff_y == 0 and diff_z == 0) or (diff_x == 0 and diff_y == 1 and diff_z == 0) or (diff_x == 0 and diff_y == 0 and diff_z == 1):
                hidden_sides += 1

        sides += 6 - hidden_sides
    
    return sides

def handle_output(sides: int):
    print(f'The answer is {sides}')


lines = read_input()
cubes = transform_input(lines)
result = solution(cubes)
handle_output(result)

The answer is 10


Тук задаваме въпросите - имаме ли нужда да раздробяваме кода чак толкова и имаме ли нужда от функция, която е само един ред ?

Отговора на двата въпроса е един и същ - зависи. Всичко опира до конкретната задача, и конкретния стил на човека - някои казват, че ако функцията е един ред, няма нужда от нея. Но пък за сметка на това, `transform_input` е по-лесно четимо от `[line_to_tuple(line) for line in lines]`.

Нека сега приложим същите идеи и върху `solution`.

In [11]:
def read_input() -> list[str]:
    line = input()
    lines = []
    while line != '':
        lines.append(line)
        line = input()

    return lines

def line_to_tuple(line: str) -> tuple[int, int, int]:
    x, y, z = line.split(',')
    return int(x), int(y), int(z)

def transform_input(lines: list[str]) -> list[tuple[int, int, int]]:
    return [line_to_tuple(line) for line in lines]

def is_side_hidden(first_cube: tuple[int, int, int], second_cube: tuple[int, int, int]) -> bool:
    diff_x = abs(first_cube[0] - second_cube[0])
    diff_y = abs(first_cube[1] - second_cube[1])
    diff_z = abs(first_cube[2] - second_cube[2])

    is_side_x_hidden = (diff_x == 1 and diff_y == 0 and diff_z == 0)
    is_side_y_hidden = (diff_x == 0 and diff_y == 1 and diff_z == 0)
    is_side_z_hidden = (diff_x == 0 and diff_y == 0 and diff_z == 1)

    return is_side_x_hidden or is_side_y_hidden or is_side_z_hidden

def count_visible_sides(cube: tuple[int, int, int], others: list[tuple[int, int, int]]) -> int:
    return 6 - sum(1 for other in others if is_side_hidden(cube, other))

def solution(cubes: list[tuple[int, int, int]]):
    visible_sides = [count_visible_sides(cube, cubes) for cube in cubes]
    return sum(visible_sides)

def handle_output(sides: int):
    print(f'The answer is {sides}')


lines = read_input()
cubes = transform_input(lines)
result = solution(cubes)
handle_output(result)

The answer is 10


Спрямо първоначалното ни решение (което бе 29 реда), това вече е 42 реда - но е доста по-четимо, доста по-лесно за промяна и доста по-тестваемо. `is_side_hidden` може да бъде написана по-кратко, но по-четимия код е по-добър от по-краткия.

### Open-closed

### Liskov-substitution

### Interface segregation

### Dependency inversion

## Clean code

### Meaningful Names

### Functions

### Comments

## PEP8

## Pylint

## Пример